In [1]:
!pip install "unsloth[kaggle-notebook]"
!pip install torch transformers datasets pandas openpyxl trl
!pip install unsloth_zoo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 28.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.7/564.7 kB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 60.8 MB/s eta 0:00:0000:010:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 272.2/272.2 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 14.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 888.1/888.1 MB 1.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.5/155.5 MB 2.6 MB/s eta 0:00:00:00:0100:

In [ ]:
import unsloth
import torch
from transformers import pipeline
from unsloth import FastLanguageModel


from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("hf_token")

if hf_token:
    from huggingface_hub import login
    try:
        login(token=hf_token, add_to_git_credential=False)
        print("Đăng nhập Hugging Face thành công!\n")
    except Exception as e:
        print(f"Lỗi đăng nhập: {e}")
        print("Kiểm tra lại token hoặc thử model công khai\n")


model_name = "VyDat/llama3-8b-multitask-10k" 

max_seq_length = 512  
dtype = None          
load_in_4bit = False  

print(f"Đang tải model {model_name}")
try:
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = model_name,
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
except Exception as e:
    print(f"LỖI: Không thể tải model. Kiểm tra lại tên repo hoặc đường dẫn: {e}")
    print("Hãy đảm bảo bạn đã đăng nhập Hugging Face (nếu model private):")
    print("Chạy 'huggingface-cli login' trong terminal của bạn.")
    exit()

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    device_map="auto",         
)

#token kết thúc
terminators = [
    pipe.tokenizer.eos_token_id,
    pipe.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

print("\n--- Bắt đầu phiên chat ---")

while True:
    try:
        user_input = input("😎 Bạn: ")
    except EOFError:
        break 

    if user_input.lower() in ['quit', 'exit']:
        print("Model: Tạm biệt! Hẹn gặp lại.")
        break
        
    if not user_input.strip():
        continue

    messages = [
        {"role": "user", "content": user_input}
    ]

    # Gọi model
    try:
        outputs = pipe(
            messages,
            max_new_tokens=512,       
            eos_token_id=terminators,
            do_sample=True,
            temperature=0.7,          
            top_p=0.9,               
        )
        
        response = outputs[0]['generated_text'][-1]['content']
        print(f"\n🤖 Model: {response.strip()}")
        print("\n\n")

    except Exception as e:
        print(f"Đã xảy ra lỗi trong quá trình inference: {e}")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-10-29 01:53:04.257431: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761702784.642094      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761702784.754196      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!
Đăng nhập Hugging Face thành công!

--- Đang tải model VyDat/llama3-8b-multitask-10k ---
==((====))==  Unsloth 2025.10.11: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

...


🤖 Model: Không có thông tin cụ thể nào về ý kiến của bạn về việc mèo.





😎 Bạn:  "tôi không thích mèo" trong ngôn ngữ kí hiệu



🤖 Model: tôi thích mèo





😎 Bạn:  "hôm nay có gì vui không" trong ngôn ngữ kí hiệu biểu diễn như theeso nào



🤖 Model: hôm nay có gì vui không



